<a href="https://colab.research.google.com/github/tristanengst/cnn-evolutionary-hyperparameters/blob/master/CNN_Playground.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#Imports
import keras
import tensorflow as tf
from keras import layers
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import random

#Presets
num_images = 2000

#Import data—data is local
data = pd.read_csv('train.csv')
images = data.iloc[0:num_images,1:]
train_images = images / 255.0
train_images = train_images.values.reshape(num_images, 28, 28, 1)
train_labels = data.iloc[0:num_images,:1]

train_images, validation_images, train_labels, validation_labels = train_test_split(train_images, train_labels, train_size = .5, random_state=42)

models = []

class Metadata:
    def __init__(self, dense_layers, dense_layer_nodes, kernels_0, kernels_1):
        self.dense_layers = dense_layers
        self.dense_layer_nodes = dense_layer_nodes
        self.kernels_0 = kernels_0
        self.kernels_1 = kernels_1

class ModelWithMetadata:
    def get_accuracy(self, model):
        try:
            model.compile(optimizer=tf.train.AdamOptimizer(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
            model.fit(train_images, train_labels, batch_size=100, epochs=10, verbose=0, validation_data=(validation_images, validation_labels))
            return model.evaluate(validation_images, validation_labels)[1]
        except:
            print("Model failed")
            return 0.0
    
    def __init__(self, metadata):
        self.model = self.get_model_from_seed(metadata)
        self.metadata = metadata
        self.accuracy = self.get_accuracy(self.model)
        print("Trained model with accuracy ", self.accuracy)
    
    #Returns a model, input is of type metadata
    def get_model_from_seed(self, metadata):
        model = keras.Sequential()
        model.add(keras.layers.Conv2D(metadata.kernels_0, 2, strides=(1,1), padding="valid", input_shape=(28, 28, 1), activation="relu"))
        model.add(keras.layers.AveragePooling2D(pool_size=(2, 2), strides=None, padding='valid',))
        model.add(keras.layers.Conv2D(metadata.kernels_1, 2, strides=(1,1), padding="valid", input_shape=(28, 28, 1), activation="relu"))
        model.add(keras.layers.AveragePooling2D(pool_size=(2, 2), strides=None, padding='valid',))
        model.add(keras.layers.Flatten())
        for i in range(metadata.dense_layers):
            model.add(keras.layers.Dense(metadata.dense_layer_nodes[i], activation="relu", use_bias=True, kernel_initializer="glorot_uniform", bias_initializer="zeros"))
        model.add(keras.layers.Dense(10, activation="softmax", use_bias=True, kernel_initializer="glorot_uniform", bias_initializer="zeros"))
        return model

def mutate(model):
    dense_layers = max(1, model.metadata.dense_layers + random.randint(-1,2))
    dense_layer_nodes = list()
    for i in range(dense_layers):
        if (i < len(model.metadata.dense_layer_nodes)):
                dense_layer_nodes.append(max(10, model.metadata.dense_layer_nodes[i] + random.randint(-20,20)))
        else:
                dense_layer_nodes.append(random.randint(10,1001))
    kernels_0 = max(10, model.metadata.kernels_0 + random.randint(-20, 20))
    kernels_1 = max(10, model.metadata.kernels_1 + random.randint(-20, 20))
    return(Metadata(dense_layers, dense_layer_nodes, kernels_0, kernels_1))

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [0]:
def evolve(num_generations):
    for i in range(num_generations):
        generation_accuracy = 0
        for i in range(10):
            models.append(ModelWithMetadata(mutate(models[0]))) #Find best model and delete the rest
            generation_accuracy = generation_accuracy + models[i].accuracy
        models.sort(key=lambda model: model.accuracy)
        while (len(models) > 1):
            models.pop(0)
        print("Best accuracy of generation was", models[0].accuracy, "Average generational accuracy:", generation_accuracy / 10)
    print(models[0].model.summary())
    print(models[0].metadata)
    return models[0].model.get_config()

models.append(ModelWithMetadata(Metadata(3, [1000, 1000, 200], 128, 32)))
evolve(10)
        
            

1000/1000 [==============================] - 1s 733us/step
Trained model with accuracy  0.937
1000/1000 [==============================] - 1s 811us/step
Trained model with accuracy  0.921
1000/1000 [==============================] - 1s 692us/step
Trained model with accuracy  0.933
1000/1000 [==============================] - 1s 660us/step
Trained model with accuracy  0.926
1000/1000 [==============================] - 1s 813us/step
Trained model with accuracy  0.926
